### Количество вопросов по каналам
### Количество ответов пользователя по каналам
### Стата по лайкам за год:
- каких лайков сколько  
- самые залайканные посты в открытых каналах (включая максимум :parrot: , :pepe_sad: , :catshake: , :ods: , ...)  

### Импорт библиотек

In [ ]:
import glob
import pandas as pd
import json
from pandas.io.json import json_normalize

from datetime import datetime

import os

import re

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

%matplotlib inline
import matplotlib.pyplot as plt



In [ ]:
print ('Размер файла', round(os.path.getsize('2018_ods_raw.csv')/1000/1000.0,2), 'мегабайт')

In [ ]:
df = pd.read_csv('2018_ods_raw.csv', sep='|', encoding='utf-8', dtype=str)

In [ ]:
# tp = pd.read_csv('2018_ods_raw.csv', sep='|', encoding='utf-8', dtype=str, iterator=True, chunksize=1000)
# print (tp)
# #<pandas.io.parsers.TextFileReader object at 0x00000000150E0048>
# df = pd.concat(tp, ignore_index=True)

In [ ]:
# df.info() 

In [ ]:
len(df)

In [ ]:
# Почему в dataframe 55к а в csv 80 !?!??!?!
!cat 2018_ods_raw.csv | wc -l

In [ ]:
# Все столбцы таблицы
# list(json_df)

In [ ]:
df.tail()

In [ ]:
# Добавляю категорию для сообщений
df['cat'] = df['FILE']
df['cat'] = df['cat'].str.replace('/opt/app/data/shared/latest_dump/', '')
df['cat'] = df['cat'].str.replace('\/.*','').str.strip()

In [ ]:
df['thread_ts'] = pd.to_datetime(df['thread_ts'], unit='s')
df['ts'] = pd.to_datetime(df['ts'], unit='s')

# Информация по сообщениям
##### Количество сообщений: 55930
##### Количество родительских сообщений: 2801
##### Количество не пустых родительских сообщений: 2750
##### Родительское сообщений набравшее больше всего ответо (246 шт.) https://opendatascience.slack.com/archives/C91N8TL83/p1542103865495600

In [ ]:
# Количество ответов на родительские сообщения 
df.groupby(['thread_ts'])['thread_ts'].agg('count').sort_values(ascending=False).head()

In [ ]:
# https://opendatascience.slack.com/archives/C91N8TL83/p1542103865495600
# df.loc[(df['thread_ts'] == '2018-11-13 10:11:05.495599985') & (df['thread_ts'] == df['ts'])]
# https://opendatascience.slack.com/archives/C91N8TL83/p1542103865495600
# df.loc[(df['thread_ts'] == '2018-11-14 09:55:01.799499989') & (df['thread_ts'] == df['ts'])]

In [ ]:
 df[df.thread_ts.isnull()]

In [ ]:
# Родительские сообщения
# Возможно неправильно!?!?!!?
# df_parent = df[df.thread_ts == df.ts] # 2801
# df_parent = df[df.thread_ts.isnull()] # 26652 из них _random_b 3491 не нашел как связать с ответами
# df_parent = df[(df.thread_ts.isnull()) | (df.thread_ts == df.ts)]

df_parent = df.loc[(df['thread_ts'] == df['ts'])]

print ('Количество сообщений:', len(df))
print ('Количество родительских сообщений:', len(df_parent))
print ('Среднее количество ответов на родительское сообщение:', round(len(df)/len(df_parent),2))

In [ ]:
# Самые популярные родительские сообщения по количеству ответов
# reply_count больше 200 почему то не бывает хотя есть сообщения с большим количеством коментариев

# df_parent[['thread_ts', 'cat', 'text', 'reply_count']].sort_values(['reply_count'], ascending=False).head()

In [ ]:
# # df_a.groupby(['cat', 'user']).size().head(10)
# df_x = df.loc[df['thread_ts'].isin(df_parent['thread_ts'])]
# # df_x[['thread_ts', 'ts', 'cat', 'text', 'reply_count']]
# df_x.groupby(['thread_ts'])['thread_ts'].agg('count').sort_values(ascending=False).head()

In [ ]:
# Удаляю сообщения с пустым text
# Почему такое бывает не разобрался
df_parent = df_parent.dropna(subset=['text'])
print ('Количество не пустых родительских сообщений:', len(df_parent))

In [ ]:
# Родительских сообщений по каналам
df_parent[['cat','thread_ts']]\
        .groupby(['cat'])['thread_ts'] \
        .count() \
        .reset_index(name='count') \
        .sort_values(['count'], ascending=False) \
        .head()

# Информация по заданным вопросам
##### Количество родительских сообщений с вопросами: 1840 из 2750


In [ ]:
# Добавляю столбец текстом сообщений в номальной форме для того что бы потом искать сообщения с вопросами
df_parent['morph_text'] = df_parent['text']

In [ ]:
def to_norm_form(data, column):
    full_words_list = []
    words_row_list = data[column].tolist()
    # Каждую строчку в переданном столбце
    for i in range(len(words_row_list)):
        # Получаю список слов
        words_list = re.sub("[^\w]", " ",  words_row_list[i]).split()
        # Каждое слово из строки
        norm_words_list = []
        for word in words_list:
            norm_word = morph.parse(word)[0].normal_form
            norm_words_list.append(norm_word)
        
        full_words_string = ' '.join(norm_words_list)
        full_words_list.append(full_words_string)
        
    return full_words_list

In [ ]:
%%time
# Запонляю столбец morph_text текстом сообщений в номальной форме для того что бы потом искать сообщения с вопросами
df_parent['morph_text'] = to_norm_form(df_parent, 'morph_text')

In [ ]:
df_parent[['text', 'morph_text']].head()

In [ ]:
# Добавляю признак что в тексте был знак вопроса
df_parent['found_question_mark'] = df_parent['text'].str.contains('\?')

In [ ]:
df_parent[['text', 'morph_text', 'found_question_mark']].tail()

In [ ]:
# morph.parse('зачем')[0].normal_form

In [ ]:
# Количество родительских сообщений с вопросами
# QUESTION_WORD_LEMMAS = ("как", "как-то", "какой", "какой-то", "зачем", "почему", "когда", "кто", "где", "когда", "куда", "куда-то", "чот")
#QUESTION_WORDS = ('вопрос', 'обьяснит', 'подсказать', 'посоветовать', 'как') # Количество родительских сообщений с вопросами: 1828
QUESTION_WORDS = ('вопрос', 'обьяснит', 'подсказать', 'посоветовать', 'как', 'почему', 'зачем')

df_q = df_parent.loc[(df_parent['morph_text'].str.contains('|'.join(QUESTION_WORDS))) | (df_parent.found_question_mark == True)].reset_index()

In [ ]:
print ('Количество родительских сообщений:', len(df_parent))
print ('Количество родительских сообщений с вопросами:', len(df_q))

In [ ]:
# Вопросов в родительских сообщениях по каналам
df_q[['cat','thread_ts']]\
        .groupby(['cat'])['thread_ts'] \
        .count() \
        .reset_index(name='count') \
        .sort_values(['count'], ascending=False) \
        .head()

In [ ]:
# Вопросов в родительских сообщениях по каналам

# df_q = df_q[df_q.cat != '_random_b'] # 5287
# df_q = df_q[df_q.cat != 'stack_overflow'] # 456

# df_q.groupby(['cat'])['thread_ts'].agg('count').sort_values(ascending=False).head()

In [ ]:
plt.rcParams["figure.figsize"] = (16, 9)

df_q.groupby(['cat'])['thread_ts'].agg('count').plot.bar()
plt.title('Количество вопросов по каналам за 2018 год (шт.)', loc='center')

plt.xlabel('Канал')
plt.ylabel('Количество')

# plt.savefig('2018-ods-questions.svg', format='svg')
plt.savefig('2018-ods-questions.png', bbox_inches = 'tight', dpi=600)
# I used 1200 dpi because a lot of scientific journals require images in 1200 / 600 / 300 dpi depending on what the image is of

# Информация по ответам на вопросы

In [ ]:
# # reply_count показывает странные цифры

# # Ответов на сообщения вопросы по полю reply_count
# # df_q[['cat', 'thread_ts', 'text', 'morph_text', 'reply_count']].sort_values('reply_count', ascending=False).head()
# df_q[['cat', 'thread_ts', 'reply_count']].sort_values('reply_count', ascending=False).head()

In [ ]:
# df_q.loc[(df_q['thread_ts'] == '2018-11-01 18:36:36.419199944')]

In [ ]:
# df.loc[df['thread_ts'].isin(df_q['thread_ts'])].tail()

In [ ]:
# Ответов по вхождению сообщений пользователей в сообщения вопросы

# наверно будет хорошей идеей отфильтровать из ответов на вопросы пользователей которые задали родительский вопрос

# df_a = df.loc[df['thread_ts'].isin(df_q['thread_ts'])]
# df_a[['cat','thread_ts', 'user']]\
#                         .groupby(['cat', 'thread_ts', 'user'])['thread_ts'] \
#                         .count() \
#                         .reset_index(name='count') \
#                         .sort_values(['count'], ascending=False) \
#                         .head()

In [ ]:
len(df_a)

In [ ]:
# df_q.loc[(df_q['thread_ts'] == '2018-11-16 08:59:45.085799932')]

In [ ]:
# Ответов по вхождению сообщений пользователей в сообщения вопросы
# наверно будет хорошей идеей отфильтровать из ответов на вопросы пользователей которые задали родительский вопрос

df_a = df.loc[df['thread_ts'].isin(df_q['thread_ts']) & ~df['user'].isin(df_q['user'])]

df_a[['cat', 'user']]\
    .groupby(['cat', 'user'])['user'] \
    .count() \
    .reset_index(name='count') \
    .sort_values(['count'], ascending=False) \
    .head()

In [ ]:
len(df_a)

In [ ]:
df_q.tail()

In [ ]:
plt.rcParams["figure.figsize"] = (16, 9)

df_a.groupby(['cat'])['cat'].agg('count').plot.bar()
plt.title('Количество ответов по каналам за 2018 год (шт.)', loc='center')

plt.xlabel('Канал')
plt.ylabel('Количество')

# plt.savefig('2018-ods-questions.svg', format='svg')
plt.savefig('2018-ods-questions.png', bbox_inches = 'tight', dpi=600)
# I used 1200 dpi because a lot of scientific journals require images in 1200 / 600 / 300 dpi depending on what the image is of

In [ ]:
users_list = json.loads(open('/opt/app/data/shared/latest_dump/users.json').read())
users_df = json_normalize(users_list)

# users_df.info()
# users_df[['id', 'name']].head()

In [ ]:
df_a = pd.merge(df_a, users_df,  how='left', left_on=['user'], right_on = ['id'])

In [ ]:
# Топ пользователей по ответам на вопросы
df_a.groupby(['user', 'name'])['user'].agg('count').reset_index(name='count').sort_values(['count'], ascending=False).head(10)

In [ ]:
plt.rcParams["figure.figsize"] = (16, 9)

df_user_replies = df_a.groupby(['user', 'name'])['user'].agg('count').reset_index(name='count').sort_values(['count'], ascending=False).head(50)
df_user_replies.set_index('name').plot.bar()

plt.title('Топ пользователей по ответам на вопросы за 2018 год (шт.)', loc='center')
plt.xlabel('Пользователи')
plt.ylabel('Количество ответов на вопросы')

# plt.savefig('2018-ods-questions.svg', format='svg')
plt.savefig('2018-ods-user-answers.png', bbox_inches = 'tight', dpi=600)
# I used 1200 dpi because a lot of scientific journals require images in 1200 / 600 / 300 dpi depending on what the image is of